# Geographic Data Visualization

In [1]:
import io
import warnings
import datetime as dt

import requests
import pandas as pd
import xarray as xr

from ipyleaflet import Map, basemaps, Heatmap, WMSLayer, Velocity

warnings.filterwarnings('ignore')

## First, some config

In [2]:
# Date to obtain data relative to same period
date = dt.datetime(2018, 10, 9)

# If this notebook is public, the key above is invalid
# So, you need get a new one in https://nrt4.modaps.eosdis.nasa.gov/profile/app-keys
active_fire_key = '276771C6-D4D4-11E8-85C6-D2089B439298'

date_active_fire = date.strftime('%Y%j')
local_active_fire = 'Global'
url_active_fire = "https://nrt4.modaps.eosdis.nasa.gov/api/v2/files/contents/FIRMS/viirs/{local}/" \
                  "VIIRS_I_{local}_VNP14IMGTDL_NRT_{date}.txt".format

date_sea_winds = date.strftime('%Y%m%d')
url_sea_winds = 'http://www.ncei.noaa.gov/thredds/dodsC/uv/daily/2000s/uv{date}rt.nc'.format

## Getting data

In order to play with geographic data we'll use VIIRS active fire data from [LANCE-NRT](https://nrt4.modaps.eosdis.nasa.gov/).

In [3]:
url = url_active_fire(local=local_active_fire, date=date_active_fire)
headers = {
    "Authorization": "Bearer {key}".format(key=active_fire_key),
}
res = requests.get(url, headers=headers)
df = pd.read_csv(io.BytesIO(res.content))
df.head()

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,confidence,version,bright_ti5,frp,daynight
0,65.32442,56.42158,296.0,0.43,0.62,2018-10-09,00:48,N,nominal,1.0NRT,269.4,0.5,N
1,66.76722,56.19038,300.9,0.36,0.57,2018-10-09,00:48,N,nominal,1.0NRT,272.8,4.2,N
2,66.68442,77.32233,317.6,0.70,0.75,2018-10-09,00:48,N,nominal,1.0NRT,269.6,3.8,N
3,66.95091,56.65474,304.7,0.35,0.57,2018-10-09,00:48,N,nominal,1.0NRT,270.7,0.9,N
4,67.17809,56.63229,329.7,0.34,0.56,2018-10-09,00:48,N,nominal,1.0NRT,271.7,2.7,N


Our next data is about sea winds. The snippet of code below retrieve [blended sea winds](https://www.ncdc.noaa.gov/data-access/marineocean-data/blended-global/blended-sea-winds) data from [NOAA](https://www.noaa.gov/) via [OPeNDAP](https://www.opendap.org/).

In [4]:
ds = xr.open_dataset(url_sea_winds(date=date_sea_winds))

## Preparing visualization

For active fire data, we show a heatmap distribution.

In [5]:
heatmap = Heatmap(locations=df[['latitude', 'longitude', 'frp']].values.tolist(), radius=20)

And for sea winds, we show how is the wind behaving with direction and velocity.

In [6]:
display_options = {
    'velocityType': 'Blended Sea Winds',
    'displayPosition': 'bottomleft',
    'displayEmptyString': 'No wind data'
}
velocity = Velocity(data=ds.isel(time=0, zlev=0),
                    zonal_speed='u',
                    meridional_speed='v',
                    latitude_dimension='lat',
                    longitude_dimension='lon',
                    velocity_scale=0.01,
                    max_velocity=20,
                    display_options=display_options)

Here, we load a [WMS](https://en.wikipedia.org/wiki/Web_Map_Service) layer with a blue marble mosaic.

In [7]:
wms = WMSLayer(
    url="https://demo.boundlessgeo.com/geoserver/ows?",
    layers="nasa:bluemarble"
)

For this to run properly, you must have installed [IPyleaflet](https://github.com/jupyter-widgets/ipyleaflet#installation)

In [8]:
m = Map(
    layers=(wms,),
    center=(0, 0),
    zoom=1
)
m.add_layer(heatmap)
m.add_layer(velocity)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …